# Hybrid Retrieval (BM25 + Dense) with RRF — Concept Overview

When a user asks a question, we need to **search through documents** (e.g., articles).

There are two popular ways to search:

- **BM25 (sparse retrieval)** — based on **exact word matching**.  
  It works great when the query and the document share the same words  
  *(e.g., “matrícula”, “plazos”, “UV”)*.

- **Dense retrieval (embeddings)** — based on **semantic meaning**.  
  Even if the words differ, it can match similar meanings  
  *(e.g., “inscripción” ≈ “matrícula”)*.

Each method fails where the other succeeds.  
The usual solution is to **combine both**, increasing recall without sacrificing precision.

# What is a Hybrid Retriever?

A **hybrid retriever** runs **two searches in parallel**:

1. One using **BM25** — ranking documents by **textual relevance**.
2. Another using **embeddings** — ranking by **semantic similarity**.

Finally, it **merges both ranked lists** into a single, more reliable ranking.

# RRF (Reciprocal Rank Fusion)

**Reciprocal Rank Fusion (RRF)** is a simple yet powerful rule for combining ranked lists.

### Inputs:
- **List A** (e.g., BM25) returns documents with ranks: `rank_A(d) = 1, 2, 3, ...` (1 = best)
- **List B** (e.g., Dense) returns ranks: `rank_B(d) = 1, 2, 3, ...`

### Formula (per document `d`):
RRF\_score(d) = 1/(k + rank_A(d)) + 1/(k + rank_B(d))


- **k** is a constant (typically `60`) — it ensures top ranks matter more but prevents one list from dominating.
- If a document doesn’t appear in a list, we treat it as having a **very large rank** (or simply no contribution).

### Intuition:
- A document ranked very high in **either** list gets a **strong boost** (`1/(k+1)`).
- A document that ranks moderately in **both** lists also gains a solid combined score.
- **RRF is robust** — no need to normalize scores or tune probabilities, just use ranks.


# Simple Example of RRF

Assume we take the **top-3** documents from each retriever:

**BM25 (List A)** → `d1`, `d2`, `d3`  
**Dense (List B)** → `d2`, `d3`, `d4`  

We set `k = 60` (a common choice).

---

### Step-by-step computation

| Document | rank_A | rank_B | RRF Score Calculation | Total |
|-----------|--------|--------|------------------------|--------|
| **d1** | 1 | — | 1/(60+1) + 0 | **1/61** |
| **d2** | 2 | 1 | 1/(60+2) + 1/(60+1) | **1/62 + 1/61** |
| **d3** | 3 | 2 | 1/(60+3) + 1/(60+2) | **1/63 + 1/62** |
| **d4** | — | 3 | 0 + 1/(60+3) | **1/63** |

---

### Final order (highest to lowest RRF score):
1. **d2** → highest combined (top in both lists)  
2. **d3** → solid mid-rank in both lists  
3. **d1** → strong in BM25 only  
4. **d4** → found only in Dense search

**d2 wins** because it performs well in *both* search types.  
**d1** was best in BM25 but missing in Dense, so it drops behind.


### 1 — Environment & Imports

In [20]:
# ===============================================
#  Environment & Imports
# ===============================================

# --- Standard library ---
import os
import re
import glob
import hashlib
from datetime import datetime
from uuid import uuid4
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import numpy as np

# --- Third-party libraries ---
from dotenv import load_dotenv
from openai import OpenAI
import chromadb
from rank_bm25 import BM25Okapi
from pypdf import PdfReader
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
)

# --- Environment setup ---
load_dotenv()


# configuration expected from .env.
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
CHROMADB_TOKEN = os.environ.get("CHROMADB_TOKEN")
CHROMA_TENANT = os.environ.get("CHROMA_TENANT")       
CHROMA_DB = os.environ.get("CHROMA_DB")             
CHROMA_HOST = os.environ.get("CHROMA_HOST")            
CHROMA_COLLECTION = os.environ.get("CHROMA_COLLECTION") 

# Early fail-fast to surface configuration errors deterministically.
assert OPENAI_API_KEY, "Set OPENAI_API_KEY"
assert CHROMADB_TOKEN and CHROMA_TENANT and CHROMA_DB and CHROMA_COLLECTION, "Set Chroma Cloud env vars"


In [2]:
# OpenAI client
llm = OpenAI(api_key=OPENAI_API_KEY)

# Chroma client (Cloud)
client = chromadb.CloudClient(
    api_key=CHROMADB_TOKEN,
    tenant=CHROMA_TENANT,
    database=CHROMA_DB,
)
collection = client.get_or_create_collection(name=CHROMA_COLLECTION)

# Minimal tokenizer dedicated to BM25: lowercasing + simple word/symbol segmentation.
# this is sufficient for BM25’s bag-of-words scoring.
def simple_tokenize(text: str) -> List[str]:
    return re.findall(r"\w+|\S", (text or "").lower())

### 2 - Embedding helpers (OpenAI)

In [16]:
EMBEDDING_MODEL = "text-embedding-3-small"  
CHROMA_GET_LIMIT = 300

In [ ]:
def embed_texts(texts: List[str]) -> List[List[float]]:
    """Vectorize a batch of texts using OpenAI embeddings. 
    Returns one embedding per input text, preserving order."""
    if not texts:
        return []
    # Send the texts to the OpenAI embedding model to generate embeddings.
    # 'resp' is an API response object containing metadata + one embedding per text.
    resp = llm.embeddings.create(model=EMBEDDING_MODEL, input=texts)
    # Extract only the embedding vectors from the response (ignore metadata).
    # resp.data is a list of objects; each has an attribute 'embedding' which is a list of floats.
    return [d.embedding for d in resp.data]

def embed_query(query: str) -> List[float]:
    """Vectorize a single query (thin wrapper over embed_texts)."""
    return embed_texts([query])[0]


### 3 — PDF Ingestion, Two-Pass Chunking, Embedding, and Upsert to Chroma

#### Scan `./data` and extract text from each PDF

For each PDF:
- read pages, extract text
- join per-document text with double newlines
- build a minimal metadata record
- keep both full document text and per-page text for traceability


In [4]:
def stable_doc_id(path: str, mtime: float) -> str:
    """
    Generate a stable, deterministic document ID based on file path and last modification time.
    This ensures identical PDFs (same path + unchanged timestamp) get the same ID across runs.
    """
    raw = f"{os.path.abspath(path)}::{mtime}".encode("utf-8")
    return hashlib.sha1(raw).hexdigest()  # 40-char hex string

In [5]:
DATA_DIR = "./data"
assert os.path.isdir(DATA_DIR), f"Data folder not found: {DATA_DIR}"

pdf_paths = sorted(glob.glob(os.path.join(DATA_DIR, "**/*.pdf"), recursive=True))
print(f"Found {len(pdf_paths)} PDFs under {DATA_DIR}")

raw_docs: List[Dict[str, Any]] = []  # items: {"doc_id","filename","path","text","pages_text","last_modified"}

for pdf_path in pdf_paths:
    try:
        reader = PdfReader(pdf_path)
    except Exception as e:
        print(f"[WARN] Skipping {pdf_path}: {e}")
        continue

    # Extract per-page text (strip leading/trailing whitespace)
    pages_text: List[str] = []
    for p in reader.pages:
        t = p.extract_text() or ""
        t = t.strip()
        if t:
            pages_text.append(t)

    # Join all page texts into a single document text
    full_text = "\n\n".join(pages_text)

    # Skip empty documents (e.g., scanned PDFs without OCR)
    if not full_text.strip():
        print(f"[WARN] Empty text: {pdf_path}")
        continue

    # Metadata
    stat = os.stat(pdf_path)
    last_modified_ts = stat.st_mtime
    last_modified = datetime.fromtimestamp(last_modified_ts).isoformat()
    doc_id = stable_doc_id(pdf_path, last_modified_ts)

    raw_docs.append({
        "doc_id": doc_id,
        "filename": os.path.basename(pdf_path),
        "path": os.path.abspath(pdf_path),
        "text": full_text,
        "pages_text": pages_text,  # retain per-page traceability
        "last_modified": last_modified,
    })

print(f"Ingested {len(raw_docs)} PDF documents with text.")

Found 25 PDFs under ./data
Ingested 25 PDF documents with text.


#### Two-pass splitting
First pass at character level preserves coarse structure and avoids awkward breaks.

Second pass at token level introduces overlap to improve recall near boundaries.

character chunks ≈ 1000 (no overlap)

token chunks ≈ 384 tokens with 64-token overlap

In [6]:
char_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0,
)

token_splitter = SentenceTransformersTokenTextSplitter(
    tokens_per_chunk=384,
    chunk_overlap=64,
)


c:\Users\Zakaria\anaconda3\envs\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Split documents into page-aware chunks with rich metadata

For each document:

iterate page by page

apply character-level splitting, then token-level splitting

construct chunks with metadata capturing the source PDF and page

In [7]:
def split_into_chunks_by_page(doc: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Produce page-aware chunks from a document.
    Pipeline: page -> character-level segments -> token-level chunks (with overlap).
    Each chunk carries metadata sufficient for page-level citation and source tracing.
    """
    chunks: List[Dict[str, Any]] = []
    pages: List[str] = doc.get("pages_text") or [doc["text"]]
    total_pages = len(pages)

    global_chunk_idx = 0  # running index of chunks across the whole document

    for page_num, page_text in enumerate(pages, start=1):
        if not page_text or not page_text.strip():
            continue

        # First pass: character-level segmentation
        char_segments = char_splitter.split_text(page_text)

        # Second pass: token-level segmentation with overlap
        page_chunk_idx = 0
        for seg in char_segments:
            token_chunks = token_splitter.split_text(seg)
            for chunk_text in token_chunks:
                # Skip degenerate chunks that are purely whitespace
                if not chunk_text.strip():
                    continue

                chunk_id = f"{doc['doc_id']}::p{page_num}::ch{global_chunk_idx}"

                chunks.append({
                    "id": chunk_id,
                    "text": chunk_text,
                    "metadata": {
                        "doc_id": doc["doc_id"],
                        "filename": doc["filename"],
                        "path": doc["path"],
                        "page": page_num,                    # 1-based page number for citation
                        "page_chunk_index": page_chunk_idx,  # chunk index within the page
                        "chunk_index": global_chunk_idx,     # global chunk index within the doc
                        "total_pages": total_pages,
                        "last_modified": doc["last_modified"],
                        # Useful anchor for viewers that support page fragments:
                        "source": f"{doc['path']}#page={page_num}",
                        # Additional domain-specific fields can be added here (e.g., "section", "language").
                    }
                })

                page_chunk_idx += 1
                global_chunk_idx += 1

    print(f"[{doc['filename']}] produced {len(chunks)} chunks with page metadata.")
    return chunks

#### Build the complete chunk list across the corpus

Aggregate page-aware chunks from all documents into a single list for embedding and upsert.

In [8]:
all_chunks: List[Dict[str, Any]] = []
for d in raw_docs:
    all_chunks.extend(split_into_chunks_by_page(d))

print(f"Total chunks prepared: {len(all_chunks)}")


[2024.findings-acl.456.pdf] produced 89 chunks with page metadata.
[2025.acl-long.366.pdf] produced 111 chunks with page metadata.
[2401.10020v3.pdf] produced 80 chunks with page metadata.
[2401.10774v3.pdf] produced 107 chunks with page metadata.
[2401.15884v3.pdf] produced 69 chunks with page metadata.
[2401.18059v1.pdf] produced 92 chunks with page metadata.
[2402.01306v4.pdf] produced 95 chunks with page metadata.
[2402.13547v2.pdf] produced 88 chunks with page metadata.
[2402.13753v1.pdf] produced 78 chunks with page metadata.
[2403.03206v1.pdf] produced 111 chunks with page metadata.
[2403.19887v2.pdf] produced 57 chunks with page metadata.
[2404.16130v2.pdf] produced 105 chunks with page metadata.
[2405.04437v3.pdf] produced 109 chunks with page metadata.
[2405.14734v3.pdf] produced 131 chunks with page metadata.
[2405.21060v1.pdf] produced 216 chunks with page metadata.
[2406.04692v1.pdf] produced 61 chunks with page metadata.
[2407.08608v2.pdf] produced 81 chunks with page met

#### Embed and upsert chunks into Chroma (batched)

Procedure:

batch the chunk list to respect rate limits

compute embeddings for each batch

upsert (ids, documents, embeddings, metadatas) into the target Chroma collection

In [9]:
BATCH = 256  # tune based on provider limits and latency characteristics

def batched(iterable, n: int):
    """Yield successive n-sized lists from iterable."""
    for i in range(0, len(iterable), n):
        yield iterable[i : i + n]

upserted = 0
for batch in batched(all_chunks, BATCH):
    texts = [c["text"] for c in batch]
    ids   = [c["id"] for c in batch]
    metas = [c["metadata"] for c in batch]

    # Embeddings (OpenAI-compatible)
    embeds = embed_texts(texts)

    # Upsert into Chroma
    collection.upsert(
        ids=ids,
        documents=texts,
        embeddings=embeds,
        metadatas=metas,
    )
    upserted += len(batch)

print(f"Upserted {upserted} chunks into Chroma")

Upserted 2922 chunks into Chroma


In [10]:
query_text = "What did Qwen2.5-1M models do to enhance training efficiency and reduce costs?"

q_embed = embed_query(query_text)  
results = collection.query(
    query_embeddings=[q_embed],
    n_results=5,
    include=["documents", "metadatas", "distances"],
)

docs      = results.get("documents", [[]])[0]
metadatas = results.get("metadatas", [[]])[0]
scores    = results.get("distances", [[]])[0]

for doc_text, meta, score in zip(docs, metadatas, scores):
    filename = meta.get("filename", "unknown.pdf")
    page     = meta.get("page", "?")
    source   = meta.get("source", "")
    print(f"Source: {filename} — page {page}  (distance={score:.4f})")
    if source:
        print(f"Link: {source}")
    print(doc_text[:400].strip(), "...\n")


Source: 2501.15383v1.pdf — page 15  (distance=0.5906)
Link: c:\Users\Zakaria\Downloads\bm25-dense-etrieval\data\2501.15383v1.pdf#page=15
its processing time from 4. 9 minutes to only 68 seconds. these improvements significantly reduce user waiting times for long - sequence tasks. compared to the open - source qwen2. 5 - 1m models, qwen2. 5 - turbo excels in short tasks and achieves competitive results on long - context tasks, while delivering shorter processing times and lower costs. consequently, it offers an excellent balance of ...

Source: 2501.15383v1.pdf — page 2  (distance=0.6577)
Link: c:\Users\Zakaria\Downloads\bm25-dense-etrieval\data\2501.15383v1.pdf#page=2
qwen2. 5 - 14b - instruct - 1m. compared to the 128k versions, these models exhibit significantly enhanced long - context capabilities. additionally, we provide an api - accessible model based on mixture of experts ( moe ), called qwen2. 5 - turbo, which offers performance comparable to gpt - 4o - mini but with longer con

### 4 — Refresh local mirror for BM25

rebulding a **local textual mirror** of the Chroma collection.  
Because BM25 requires access to raw text (not embeddings), we:
- Download all documents and metadata from Chroma in safe batches.
- Tokenize each text to prepare for BM25 indexing.
- Reconstruct a fresh BM25 model over the same chunk-level data stored in Chroma.

This mirror should be refreshed whenever the collection content changes  
(e.g., new PDFs ingested or existing ones updated).

In [ ]:
def fetch_all_docs_from_chroma(
    collection,
    batch_size: int = CHROMA_GET_LIMIT,
    include: Optional[List[str]] = None,
) -> Dict[str, List[Any]]:
    """
    Download all documents from a Chroma collection safely in batches.
    Returns a dictionary with three parallel lists: 'ids', 'documents', and 'metadatas'.
    This ensures a full local snapshot suitable for BM25 indexing.
    """
    include = include or ["documents", "metadatas"]

    # Initialize accumulators for all records
    all_ids, all_docs, all_metas = [], [], []
    offset = 0

    while True:
        # Fetch a batch of records from Chroma (using pagination)
        res = collection.get(include=include, limit=batch_size, offset=offset)
        ids = res.get("ids", [])
        if not ids:
            # Exit loop when no more data is returned
            break

        # Retrieve corresponding documents and metadata
        docs = res.get("documents", [[]])
        metas = res.get("metadatas", [[]])

        # Extend local buffers with the newly fetched data
        all_ids.extend(ids)
        all_docs.extend(docs)
        all_metas.extend(metas)

        # Move offset forward for the next batch
        offset += len(ids)

    # Return a unified snapshot of all data retrieved from Chroma
    return {"ids": all_ids, "documents": all_docs, "metadatas": all_metas}


# ---- Execute the full refresh process ----

# Fetch all chunk texts and metadata from Chroma Cloud
snapshot = fetch_all_docs_from_chroma(
    collection,
    batch_size=CHROMA_GET_LIMIT,
    include=["documents", "metadatas"],
)

# Unpack results into aligned lists
all_ids   = snapshot["ids"]
all_texts = snapshot["documents"]
all_metas = snapshot["metadatas"]

# Tokenize the entire corpus (chunk-level texts)
tokenized_corpus = [simple_tokenize(txt or "") for txt in all_texts]

# Rebuild a new BM25 index from the tokenized corpus
bm25 = BM25Okapi(tokenized_corpus)

# Confirmation log
print(f"BM25 snapshot rebuilt | chunks={len(all_ids)} | batch_size={CHROMA_GET_LIMIT}")

BM25 snapshot rebuilt | chunks=2922 | batch_size=300


**BM25** (short for Best Matching 25) is a classic keyword-based retrieval algorithm widely used in information retrieval systems. It provides a mathematical way to evaluate how relevant a document is to a given text query based purely on word overlap and term importance.

**What does `bm25 = BM25Okapi(tokenized_corpus)` do?**

This line builds a word-level index over a list of tokenized documents (in this case, all text chunks extracted from the PDFs). `tokenized_corpus` is a list of chunks, each represented as a list of words (tokens):

<ul>
    ["rag", "retrieval", "augmentation", "generation"]<br>
    ["graphrag", "entity", "graph", "relations"]<br>
    ["raptor", "hierarchical", "clustering", "retrieval"]<br>
    ...
</ul>

Each chunk corresponds to a segment of text (often a paragraph or a few sentences) that was previously split and stored in Chroma.

**How BM25 works internally**

*Term Frequency (TF):* BM25 counts how many times each word appears in a chunk. Words that appear more often within a chunk contribute more to its score.

*Inverse Document Frequency (IDF):* Words that appear in many chunks are considered less informative (e.g., "model", "paper"), while rare words like "GraphRAG" or "RAPTOR" get higher weight. This helps BM25 prioritize technical or distinctive terms.

*Length Normalization:* Longer chunks naturally contain more words, so BM25 normalizes scores to prevent long texts from dominating the ranking unfairly.

*Scoring:* When a query is given (e.g., "GraphRAG entity extraction"), BM25 compares the query words with the words in each chunk and assigns a relevance score. The more important and frequent the overlapping terms are, the higher the score.

In [ ]:
def bm25_topk(query: str, k: int = 5):
    """Return the top-k most relevant chunks according to BM25."""
    # 1) Tokenize the query into simple words
    tokens = simple_tokenize(query)
    
    # 2) Compute BM25 scores for all chunks
    scores = bm25.get_scores(tokens)
        
    # 3) Sort results by descending score
    top_idx = np.argsort(scores)[::-1][:k]
    
    # 4) Build structured results
    results = []
    for i in top_idx:
        results.append({
            "id": all_ids[i],
            "text": all_texts[i],
            "metadata": all_metas[i],
            "score": float(scores[i]),
        })
    return results

In [48]:
query = "What did Qwen2.5-1M models do to enhance training efficiency and reduce costs ?"
results = bm25_topk(query, k=3)

for r in results:
    meta = r["metadata"]
    print(f"Source: {meta.get('filename')} — page {meta.get('page')}")
    print(f"Score: {r['score']:.2f}")
    print(r["text"][:400].strip(), "\n---\n")

Source: 2501.15383v1.pdf — page 3
Score: 49.31
table 1 : model architecture and license of qwen2. 5 - 1m open - weight models. models layers heads ( q / kv ) tie embedding context / generation length license 7b 28 28 / 4 no 1m / 8k apache 2. 0 14b 48 40 / 8 no 1m / 8k apache 2. 0 3 pre - training long - context pre - training is computationally intensive and can be expensive. to enhance training efficiency and reduce costs, we focus on optimiz 
---

Source: 2501.15383v1.pdf — page 2
Score: 47.86
qwen2. 5 - 14b - instruct - 1m. compared to the 128k versions, these models exhibit significantly enhanced long - context capabilities. additionally, we provide an api - accessible model based on mixture of experts ( moe ), called qwen2. 5 - turbo, which offers performance comparable to gpt - 4o - mini but with longer context, stronger capabilities, and more competitive pricing. beyond the models 
---

Source: 2501.15383v1.pdf — page 15
Score: 47.31
its processing time from 4. 9 minutes to only

#### How BM25 Retrieves Metadata

BM25 itself only indexes and scores **text**, not metadata.

When `bm25 = BM25Okapi(tokenized_corpus)` is created, it builds a word-based index over the text chunks. However, metadata like filename, page, and doc_id are stored in **parallel lists**:

```python
all_ids, all_texts, all_metas
```

Each index `i` refers to the same chunk across these lists. So when BM25 returns the best matches by index (`top_idx`), those indices are used to retrieve both the text and its metadata:

```python
for i in top_idx:
    text = all_texts[i]
    meta = all_metas[i]
```

That's why the BM25 results can include information like:

```
Source: 2502.11371v1.pdf — page 2
Score: 22.24
```

Even though BM25 doesn't store metadata itself, the alignment by list position makes it possible to combine scores, text, and metadata seamlessly.

#### 5 — Dense search (Chroma)
We query the Chroma collection using OpenAI embeddings for the **query**.
Returns top-k hits with their ranks and useful metadata.
5 — Dense search over Chroma (Code)

In [42]:
def dense_search(query: str, k: int = 30) -> List[Dict[str, Any]]:
    """
    Retrieve the top-k semantically relevant chunks from Chroma.
    
    Returns a list of dicts (one per hit) with a uniform schema:
      - id:        chunk identifier (must match BM25 mirror to enable fusion)
      - text:      chunk text
      - metadata:  original metadata (filename, page, path, etc.)
      - rank:      1-based rank within this retriever
      - source:    'dense' (tag for downstream fusion)
      - score:     optional monotonic score for diagnostics (higher is better)
      - distance:  raw distance from Chroma (lower is better for L2)
    
    Notes:
    - 'k' may be tuned based on corpus size (typical values: 20–50).
    - Uses query_embeddings (client-side) to avoid dimension mismatches.
    """
    q_emb = embed_query(query)  # 1536-dim with text-embedding-3-small
    res = collection.query( # result already sorted by distance
        query_embeddings=[q_emb],
        n_results=k,
        include=["documents", "metadatas", "distances"],
    )

    hits: List[Dict[str, Any]] = []
    ids   = res.get("ids", [[]])[0] # the [0] -> first query only because chroma supports multiple queries
    docs  = res.get("documents", [[]])[0]
    metas = res.get("metadatas", [[]])[0]
    dists = res.get("distances", [[]])[0]

    for i in range(len(ids)):
        dist = float(dists[i])
        hits.append({
            "id": ids[i],
            "text": docs[i],
            "metadata": metas[i],
            "rank": i + 1,           # 1-based
            "source": "dense",
            "score": -dist,          # invert distance so higher is better (diagnostic)
            "distance": dist,        # raw distance (L2 or similar)
        })
    return hits

#### 6 — BM25 Lexical Search (Local Mirror)

The function below queries the BM25 index built over the local mirror of chunk texts
(all_texts, all_metas, all_ids). It returns the same schema as dense_search,
ensuring both retrievers can be fused consistently.

In [37]:
def bm25_search(query: str, k: int = 30) -> List[Dict[str, Any]]:
    """
    Retrieve the top-k lexically relevant chunks using the local BM25 index.
    
    Returns a list of dicts (one per hit) with the same schema as 'dense_search':
      - id, text, metadata, rank (1-based), source='bm25', score (BM25), ...
    
    Assumes:
      - 'bm25' is a BM25Okapi instance built over tokenized 'all_texts'
      - 'all_ids', 'all_texts', 'all_metas' are parallel lists with identical order
    """
    tokens = simple_tokenize(query)
    scores = bm25.get_scores(tokens)  # one score per chunk (aligned with all_texts)
    if len(scores) == 0:
        return []

    top_idx = np.argsort(scores)[::-1][:k]  # highest scores first
    hits: List[Dict[str, Any]] = []
    for r, i in enumerate(top_idx, start=1):
        hits.append({
            "id": all_ids[i],
            "text": all_texts[i],
            "metadata": all_metas[i],
            "rank": r,               # 1-based
            "source": "bm25",
            "score": float(scores[i])  # higher is better for BM25
        })
    return hits

#### 7 — Reciprocal Rank Fusion (RRF)

RRF combines ranked lists from different retrievers (BM25 + Dense) into a single robust ranking,
without needing to normalize scores. It uses ranks only, which makes it simple and effective.

In [38]:
def rrf_fuse(
    bm25_hits: List[Dict[str, Any]],
    dense_hits: List[Dict[str, Any]],
    k: int = 10,
    k_rrf: int = 60,
) -> List[Dict[str, Any]]:
    """
    Fuse BM25 and Dense ranked lists via Reciprocal Rank Fusion (RRF).
    
    RRF_score(d) = 1/(k_rrf + rank_bm25(d)) + 1/(k_rrf + rank_dense(d))
    
    Inputs:
      - bm25_hits, dense_hits: lists with uniform schema (id, text, metadata, rank, source, ...)
      - k: number of fused results to return
      - k_rrf: typical constant (e.g., 60). Larger values reduce the dominance of top-1 items.
    
    Output:
      - A list of fused results sorted by 'rrf_score' (desc), each containing:
          id, text, metadata, rrf_score, and 'sources' with per-retriever ranks.
    """
    # Build maps: id -> rank (1-based)
    rank_bm25  = {h["id"]: h["rank"] for h in bm25_hits}
    rank_dense = {h["id"]: h["rank"] for h in dense_hits}

    # Universe of candidate IDs from both retrievers
    all_ids_set = set(rank_bm25.keys()) | set(rank_dense.keys())

    fused: List[Dict[str, Any]] = [] 
    for _id in all_ids_set:
        r_b = rank_bm25.get(_id) # returns None if _id not in bm25
        r_d = rank_dense.get(_id)

        # Compute RRF score
        rrf_score = 0.0
        if r_b is not None:
            rrf_score += 1.0 / (k_rrf + r_b)
        if r_d is not None:
            rrf_score += 1.0 / (k_rrf + r_d)

        # Find the first result dictionary (h) whose "id" matches _id.
        # The next() function scans the list and returns the first match, or None if not found.
        # Prefer the version from dense_hits; if not present there, take it from bm25_hits.
        rep = (
            next((h for h in dense_hits if h["id"] == _id), None) 
            or next((h for h in bm25_hits if h["id"] == _id), None)
        )

        fused.append({
            "id": _id,
            "text": rep["text"],
            "metadata": rep["metadata"],
            "rrf_score": rrf_score,
            "sources": {
                "bm25_rank": r_b,
                "dense_rank": r_d,
            },
        })

    # Sort by fused score and truncate
    fused.sort(key=lambda x: x["rrf_score"], reverse=True)
    return fused[:k]


#### 8 — Hybrid Search Helper 

A convenience wrapper that runs both retrievers, fuses with RRF, and returns structured outputs.
The example illustrates how to call it and print page-level citations.

In [39]:
def hybrid_search(query: str, k_dense: int = 30, k_bm25: int = 30, k_final: int = 10) -> Dict[str, Any]:
    """
    Run both retrieval modes (dense + BM25) and fuse results via RRF.
    
    Returns a dict with:
      - 'bm25_hits': top-k BM25 results
      - 'dense_hits': top-k Dense results
      - 'fused': RRF-fused top-k_final results
    """
    bm25_hits  = bm25_search(query, k=k_bm25)
    dense_hits = dense_search(query, k=k_dense)
    fused      = rrf_fuse(bm25_hits, dense_hits, k=k_final, k_rrf=60)
    return {"bm25_hits": bm25_hits, "dense_hits": dense_hits, "fused": fused}

##### 9 — Example

In [49]:
query = "What did Qwen2.5-1M models do to enhance training efficiency and reduce costs ?"
result = hybrid_search(query, k_dense=30, k_bm25=30, k_final=5)

print("\n" + "=" * 110)
print(f"  QUERY: {query}")
print("=" * 110)
print(f"{'RANK':<5} {'PDF FILE':<40} {'PAGE':<6} {'RRF SCORE':<10} {'BM25':<6} {'DENSE':<6}  SOURCE")
print("-" * 110)

for i, h in enumerate(result["fused"], start=1):
    m = h["metadata"] or {}
    filename = m.get("filename", "unknown.pdf")
    page = m.get("page", "?")
    rrf = h.get("rrf_score", 0.0)
    bm25_r = (h.get("sources") or {}).get("bm25_rank", None)
    dense_r = (h.get("sources") or {}).get("dense_rank", None)

    # Prefer 'source' if present (usually "path#page=N"), else build it from 'path' + page.
    source = m.get("source")
    if not source:
        path = m.get("path", "")
        source = f"{path}#page={page}" if path else "(no source link)"

    # Clean preview (single line, truncated)
    text_preview = (h["text"][:300].strip().replace("\n", " ") + "...")

    # Row with columns
    print(f"{i:<5} {filename:<40} {str(page):<6} {rrf:<10.4f} {str(bm25_r or '-'): <6} {str(dense_r or '-'): <6}  {source}")
    # Indented preview
    print(f"        {text_preview}\n")

print("=" * 110)
print(f"Total results displayed: {len(result['fused'])}")
print("=" * 110)


  QUERY: What did Qwen2.5-1M models do to enhance training efficiency and reduce costs ?
RANK  PDF FILE                                 PAGE   RRF SCORE  BM25   DENSE   SOURCE
--------------------------------------------------------------------------------------------------------------
1     2501.15383v1.pdf                         15     0.0323     3      1       c:\Users\Zakaria\Downloads\bm25-dense-etrieval\data\2501.15383v1.pdf#page=15
        its processing time from 4. 9 minutes to only 68 seconds. these improvements significantly reduce user waiting times for long - sequence tasks. compared to the open - source qwen2. 5 - 1m models, qwen2. 5 - turbo excels in short tasks and achieves competitive results on long - context tasks, while d...

2     2501.15383v1.pdf                         2      0.0323     2      2       c:\Users\Zakaria\Downloads\bm25-dense-etrieval\data\2501.15383v1.pdf#page=2
        qwen2. 5 - 14b - instruct - 1m. compared to the 128k versions, these models ex

# 10 - Analysis of Retrieval Performance Across Methods

## Context

All three retrieval pipelines were tested on the same query:

**"What did Qwen2.5-1M models do to enhance training efficiency and reduce costs?"**

The relevant information appears explicitly on page 3 of the paper `2501.15383v1.pdf`, in the sentence:

> "To enhance training efficiency and reduce costs, we focus on optimizing data efficiency and refining training strategies during the pre-training process of Qwen2.5-1M models."

However, the three retrieval modes produced slightly different rankings for this passage.

---

## 1. Dense Retrieval (Embedding-based)

### Mechanism

- Uses OpenAI embeddings (`text-embedding-3-small`) to represent both query and chunks in a 1536-dimensional semantic space.
- Measures semantic similarity between vectors (lower distance = higher similarity).

### Result

- Top results were semantically related but not always lexically exact.
- Pages 15 and 2 dominated because they discussed training cost and efficiency but in broader contexts (e.g., latency, Turbo model performance).
- The truly exact sentence on page 3 appeared around rank 4.

### Interpretation

Dense retrieval captures meaning, not exact wording. It tends to surface conceptually similar text, even when it doesn't include the same phrase.

---

## 2. BM25 Lexical Retrieval

### Mechanism

- Uses term frequency and inverse document frequency (TF–IDF-like weighting).
- Rewards exact word overlap between query and text.

### Result

- Page 3 ranked #1, because it contains the exact phrase "enhance training efficiency and reduce costs".
- Pages 2 and 15 followed closely, since they reuse similar wording (training efficiency, cost reduction).

### Interpretation

BM25 excels at finding exact lexical matches. It is precise for direct phrasing but can miss paraphrases or semantically related content.

---

## 3. Hybrid Retrieval (RRF Fusion of Dense + BM25)

### Mechanism

- Combines the semantic ranking (Dense) and lexical ranking (BM25) using Reciprocal Rank Fusion (RRF).
- Each document's final score increases if it ranks highly in either method, and even more if in both.

### Result

- The fused top 5 were all relevant, showing stronger overlap between both systems.
- However, page 3 sometimes dropped to rank #3, because Dense retrieval favored semantically similar paragraphs about "efficiency" and "cost" that used different wording.

### Interpretation

RRF fusion increases robustness but may still overvalue semantically broad hits unless weights are adjusted or lexical matches are boosted.

future things to do: 
1) Weighted RRF
    w_bm25: float = 1.5,  
    w_dense: float = 1.0
2) Phrase/Proximity boost (mini re-ranker posterior a RRF)
3) Ajuste de chunking